In [171]:
import torch
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

from IPython.display import Image
from torchvision.utils import save_image

mnist = MNIST(root='./../data', 
              train=True, 
              download=True,
              transform=Compose([ToTensor(), Normalize(mean=(0.5,), std=(0.5,))]))

mnist_test = MNIST(root='./../data', 
              train=False, 
              download=True,
              transform=Compose([ToTensor(), Normalize(mean=(0.5,), std=(0.5,))]))

In [172]:
image_size = 784
hidden_size_1 = 128
hidden_size_2 = 64
latent_size = 64

batch_size = 100
data_loader = DataLoader(mnist, batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size, shuffle=True)

In [173]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

D = Classifier()

D1 = nn.Sequential(
    nn.Linear(image_size, hidden_size_1),
    nn.ReLU(),
    nn.Linear(hidden_size_1, hidden_size_2),
    nn.ReLU(),
    nn.Linear(hidden_size_2, 10),
    nn.Sigmoid())

In [174]:
criterion = nn.NLLLoss()
d_optimizer = optim.Adam(D.parameters(), lr=0.003)

In [197]:
def train(model, train_loader, optimizer, epoch, device = 'cpu'):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.view(len(data), 784)
        optimizer.zero_grad()
        output = model.forward(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [198]:
def test(model, test_loader, device='cpu'):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.view(len(data), 784)
            output = model.forward(data)
            test_loss += F.nll_loss(output.log(), target).item() # sum up batch loss
            ps = F.softmax(output, dim=1) * 100
            lbls = torch.zeros(len(ps), 1).to(device)
            for index, out in enumerate(ps):
                if torch.gt(out, 20).sum() > 0:
                    lbls[index] = 1
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
    
    return lbls

In [199]:
for epoch in range(4):
    train(D, data_loader, d_optimizer, epoch)
    test(D, test_loader)

torch.Size([100, 10]) torch.Size([100])
tensor([[-2.5877e+01, -1.5729e+01, -1.6407e+01, -6.2821e-05, -2.6628e+01,
         -9.8125e+00, -3.0723e+01, -1.5878e+01, -1.2426e+01, -1.2506e+01],
        [-1.1984e+01, -1.4478e+01, -1.0716e+01, -8.0357e+00, -5.8841e+00,
         -7.1898e+00, -1.7421e+01, -8.1758e+00, -7.6646e+00, -4.6516e-03],
        [-1.2686e+01, -1.6784e+01, -1.3882e+01, -1.0577e+01, -1.4765e+01,
         -1.2656e-03, -8.7875e+00, -1.6615e+01, -6.8784e+00, -9.8600e+00],
        [-1.1622e+01, -9.7206e+00, -3.9926e+00, -5.6335e+00, -1.1336e+01,
         -9.1655e+00, -1.9698e+01, -2.3857e-02, -7.3425e+00, -7.2453e+00],
        [-1.3812e+01, -1.3023e+01, -1.2261e+01, -7.7412e+00, -1.2322e+01,
         -9.8934e+00, -1.1927e+01, -1.5125e+01, -5.4976e-04, -9.9984e+00],
        [-1.5802e+01, -9.9013e+00, -1.0892e+01, -9.6795e-02, -8.6169e+00,
         -3.9850e+00, -1.3429e+01, -1.0054e+01, -2.8642e+00, -4.1139e+00],
        [-1.0621e+01, -7.7737e+00, -1.9375e+00, -1.2949e+00, -1.00

torch.Size([100, 10]) torch.Size([100])
tensor([[-1.3670e+01, -1.7334e+01, -1.1300e+01, -1.3518e+01, -6.9533e+00,
         -9.4292e+00, -2.2086e+01, -1.1649e+01, -1.3659e+01, -1.0612e-03],
        [-1.0002e+01, -7.0244e+00, -3.7243e-01, -5.0210e+00, -1.2196e+01,
         -8.8504e+00, -1.5977e+01, -1.1959e+00, -7.2775e+00, -9.0139e+00],
        [-1.0966e+01, -5.4324e-03, -6.5523e+00, -1.0467e+01, -9.3242e+00,
         -8.1686e+00, -7.2855e+00, -6.2262e+00, -7.0520e+00, -1.0027e+01],
        [-1.2999e+01, -1.7152e+01, -1.0994e+01, -1.1015e+01, -1.5307e+01,
         -7.8909e+00, -1.8284e+01, -1.2017e+01, -5.6990e-04, -8.7804e+00],
        [-1.4287e+01, -1.9911e+01, -1.4958e+01, -1.8387e+01, -1.7190e+01,
         -1.4253e+01, -8.8214e-06, -2.0895e+01, -1.1845e+01, -2.1858e+01],
        [-1.4318e+01, -1.2503e+01, -9.8618e+00, -1.3752e+01, -2.7635e-03,
         -9.4693e+00, -1.2291e+01, -1.0440e+01, -1.0217e+01, -5.9699e+00],
        [-1.4164e-03, -2.4622e+01, -6.5758e+00, -1.5712e+01, -1.50

torch.Size([100, 10]) torch.Size([100])
tensor([[-2.1756e+01, -1.5087e+01, -1.3375e+01, -1.3720e-04, -2.3629e+01,
         -8.9104e+00, -2.5075e+01, -2.2117e+01, -1.6239e+01, -1.4983e+01],
        [-4.7987e-02, -1.1783e+01, -3.3258e+00, -9.0084e+00, -8.8551e+00,
         -8.0166e+00, -5.3523e+00, -1.0045e+01, -5.2595e+00, -8.0215e+00],
        [-8.7297e+00, -1.0872e+01, -8.8391e+00, -4.0596e+00, -8.9243e+00,
         -4.4176e+00, -1.0181e+01, -8.9206e+00, -1.0266e-01, -2.6939e+00],
        [-1.6660e+01, -3.8342e-04, -1.1293e+01, -1.2920e+01, -9.3840e+00,
         -1.1941e+01, -9.8383e+00, -1.0219e+01, -8.6795e+00, -1.0930e+01],
        [-1.1132e+01, -1.2554e+01, -1.0524e+01, -1.2862e+01, -1.2317e+01,
         -7.2341e+00, -1.5851e-03, -1.7305e+01, -7.1184e+00, -1.6121e+01],
        [-1.0539e+01, -1.0441e+01, -5.7988e+00, -1.0701e+01, -1.2086e+01,
         -1.0677e+01, -1.2141e+01, -9.1823e+00, -3.2552e-03, -1.2556e+01],
        [-5.8967e-04, -2.0377e+01, -7.5406e+00, -1.4066e+01, -1.18

torch.Size([100, 10]) torch.Size([100])
tensor([[-2.2234e+01, -1.4802e+01, -4.8876e-06, -1.5575e+01, -2.6312e+01,
         -1.8895e+01, -2.5012e+01, -1.2339e+01, -1.8871e+01, -2.3367e+01],
        [-3.3497e-05, -2.2691e+01, -1.0598e+01, -1.8598e+01, -1.5035e+01,
         -2.0871e+01, -1.2275e+01, -1.7877e+01, -1.7286e+01, -1.2498e+01],
        [-1.2179e+01, -1.1664e+01, -6.4598e+00, -6.0969e+00, -1.2475e+01,
         -9.0015e+00, -2.0321e+01, -4.1476e+00, -2.2274e-02, -6.0878e+00],
        [-1.1456e+01, -1.8510e+01, -1.4589e+01, -1.6615e+01, -2.2712e+01,
         -3.8804e-03, -7.0023e+00, -1.9654e+01, -5.8256e+00, -1.3799e+01],
        [-1.1455e+01, -1.1588e+01, -9.2389e+00, -1.3877e+01, -1.0036e-01,
         -8.7141e+00, -1.5050e+01, -8.0117e+00, -1.0636e+01, -2.3555e+00],
        [-1.0252e-05, -2.1783e+01, -1.2605e+01, -1.9776e+01, -1.6538e+01,
         -1.9032e+01, -1.4701e+01, -1.5907e+01, -2.0697e+01, -1.1971e+01],
        [-1.8553e+01, -1.3494e+01, -1.1885e+01, -1.8309e-04, -1.84

tensor([[-1.9436e+01, -1.3351e-05, -1.5070e+01, -2.0499e+01, -1.2745e+01,
         -1.6082e+01, -1.5082e+01, -1.1853e+01, -1.4425e+01, -1.3044e+01],
        [-1.2052e+01, -1.5725e+01, -1.0313e+01, -1.4214e+01, -3.9268e+00,
         -8.6339e+00, -1.9777e+01, -8.9384e+00, -1.3241e+01, -2.0261e-02],
        [-7.8453e+00, -7.4673e+00, -7.9987e+00, -7.2195e+00, -8.2794e+00,
         -6.4684e+00, -9.3208e+00, -8.7979e+00, -6.3850e-03, -6.0803e+00],
        [-1.4877e+01, -3.0994e-05, -1.5991e+01, -2.0849e+01, -1.2453e+01,
         -1.3301e+01, -1.0793e+01, -1.3038e+01, -1.3350e+01, -1.4084e+01],
        [-2.1952e+01, -1.6523e+01, -1.1109e+01, -1.1917e+01, -2.0255e+01,
         -1.6791e+01, -4.0447e+01, -6.4848e-05, -1.6006e+01, -1.0055e+01],
        [-1.9833e+01, -2.2409e+01, -1.5502e+01, -1.7963e+01, -1.0201e+01,
         -1.3800e+01, -3.2122e+01, -1.1147e+01, -1.3744e+01, -5.3881e-05],
        [-5.4797e-04, -1.4843e+01, -9.4660e+00, -1.4320e+01, -1.1962e+01,
         -1.2010e+01, -7.9496e+0

...

tensor([[-1.4844e+01, -1.8584e+01, -1.7237e+01, -1.3892e+01, -2.5088e+01,
         -3.9745e-02, -9.7051e+00, -2.3064e+01, -3.2467e+00, -1.7989e+01],
        [-2.1406e+01, -1.5101e+01, -1.2439e+01, -1.3375e+01, -1.7005e+01,
         -1.7922e+01, -3.8309e+01, -5.8292e-05, -1.6779e+01, -9.8569e+00],
        [-2.2259e+01, -1.0445e+01, -3.3736e-05, -1.3622e+01, -2.0175e+01,
         -1.5532e+01, -1.9573e+01, -1.2642e+01, -1.8244e+01, -2.1614e+01],
        [-2.0469e+01, -1.0421e+01, -9.5893e+00, -1.1873e+01, -1.5535e+01,
         -1.8191e+01, -3.4780e+01, -1.0645e-04, -1.7966e+01, -1.3822e+01],
        [-1.8815e+01, -2.8079e+01, -1.9231e+01, -2.2761e+01, -2.2277e+01,
         -2.3445e+01, -4.2176e+01, -1.4305e-06, -2.5424e+01, -1.3497e+01],
        [-2.0501e+01, -1.5998e+01, -2.3842e-06, -1.3833e+01, -2.1275e+01,
         -1.7548e+01, -2.2518e+01, -1.3501e+01, -1.7591e+01, -2.1613e+01],
        [-1.2580e+01, -1.2541e+01, -1.0333e+01, -1.3830e+01, -1.2669e+01,
         -9.6644e+00, -3.2575e-0

torch.Size([100, 10]) torch.Size([100])
tensor([[-4.7465e+01, -3.4154e+01, -3.5922e+01,  0.0000e+00, -4.4267e+01,
         -2.5754e+01, -5.6907e+01, -3.5983e+01, -2.2812e+01, -2.4480e+01],
        [-2.7249e+01, -1.7327e+01, -1.1921e-07, -1.7304e+01, -2.4813e+01,
         -2.0020e+01, -2.5248e+01, -1.6612e+01, -2.4882e+01, -2.7181e+01],
        [-1.3101e+01, -1.5888e+01, -1.3397e+01, -5.0681e+00, -9.9014e+00,
         -7.0623e+00, -2.0048e+01, -8.9918e+00, -6.5160e+00, -8.8477e-03],
        [-1.7527e+01, -1.1212e+01, -2.6652e-04, -8.3557e+00, -1.7794e+01,
         -1.4293e+01, -1.7758e+01, -1.1651e+01, -1.1661e+01, -1.9056e+01],
        [-2.4803e+01, -1.4807e+01, -1.4543e-05, -1.3004e+01, -2.4091e+01,
         -2.0012e+01, -2.7149e+01, -1.1340e+01, -1.8982e+01, -2.4774e+01],
        [-2.7065e+01, -2.0046e+01, -2.1538e+01, -2.3842e-06, -2.1813e+01,
         -1.8866e+01, -3.1917e+01, -2.6112e+01, -1.3175e+01, -1.4488e+01],
        [-7.8717e+00, -8.3501e+00, -6.7075e+00, -7.9002e+00, -1.65

torch.Size([100, 10]) torch.Size([100])
tensor([[-1.3648e+01, -8.9918e+00, -5.4011e+00, -4.6496e+00, -1.1518e+01,
         -1.2920e+01, -2.5153e+01, -1.5028e-02, -7.8443e+00, -8.0828e+00],
        [-1.3638e+01, -1.4920e+01, -1.2616e+01, -7.3591e+00, -7.8945e+00,
         -6.0550e+00, -1.8664e+01, -8.9316e+00, -7.7845e+00, -3.9165e-03],
        [-1.5152e+01, -1.3454e+01, -1.2950e+01, -6.6175e+00, -1.5215e+01,
         -1.2380e+01, -1.5202e+01, -1.6398e+01, -1.3570e-03, -1.1463e+01],
        [-4.5591e-01, -7.9303e+00, -5.6684e+00, -7.4933e+00, -5.6431e+00,
         -5.5086e+00, -1.0554e+00, -7.9586e+00, -5.5245e+00, -6.3412e+00],
        [-2.6480e+01, -2.0335e+01, -1.3339e+01, -1.0995e+01, -2.1644e+01,
         -2.1142e+01, -4.7615e+01, -1.9908e-05, -2.0377e+01, -1.3411e+01],
        [-2.9772e+01, -3.0249e+01, -1.6518e+01, -1.4965e+01, -2.7794e+01,
         -2.6588e+01, -5.9734e+01, -8.3446e-07, -2.3589e+01, -1.4639e+01],
        [-1.4376e+01, -1.2955e+01, -7.7984e+00, -3.9495e+00, -1.32

torch.Size([100, 10]) torch.Size([100])


KeyboardInterrupt: 

In [178]:
G = nn.Sequential(
    nn.Linear(latent_size, 64),
    nn.ReLU(),
    
    nn.Linear(64, 128),
    nn.ReLU(),
    nn.Linear(128, 256),
    nn.ReLU(),
    nn.Linear(256, 784),
    nn.Tanh())

g_optimizer = optim.Adam(D.parameters(), lr=0.003)

In [179]:
import os

sample_dir = './../data/samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [208]:
sample_vectors = torch.randn(batch_size, latent_size)

def save_fake_images(index):
    fake_images = G(sample_vectors)
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    fake_fname = 'fake_images-{0:0=4d}.png'.format(index)
    print('Saving', fake_fname)
    save_image(fake_images, os.path.join(sample_dir, fake_fname), nrow=10)

In [209]:
b_criterion = nn.BCELoss()

def train_generator(device = 'cpu'):
    z = torch.randn(32, latent_size).to(device)
    
    for param in G.parameters():
        param.requires_grad = True
    
    fake_images = G(z)
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    
    fake_data = []
    for index, img in enumerate(fake_images):
        fake_data.append([img, 0])
    fake_loader = torch.utils.data.DataLoader(fake_data, batch_size=32, shuffle=False, num_workers=4)
    
    labels = torch.ones(32).to(device)
    
    g_optimizer.zero_grad()
    lbls = test(D, fake_loader)
    lbls.requires_grad = True
    g_loss = b_criterion(lbls, labels)
    g_loss.backward()
    g_optimizer.step()
    return g_loss, fake_images

#%%time

num_epochs = 300
d_losses, g_losses, real_scores, fake_scores = [], [], [], []

for epoch in range(num_epochs):
    g_loss, fake_images = train_generator()
    save_fake_images(epoch+1)


Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0001.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0002.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0003.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0004.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0005.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0006.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0007.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0008.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0009.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0010.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0011.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0012.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

S


Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0105.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0106.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0107.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0108.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0109.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0110.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0111.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0112.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0113.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0114.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0115.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0116.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

S


Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0209.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0210.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0211.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0212.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0213.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0214.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0215.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0216.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0217.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0218.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0219.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

Saving fake_images-0220.png

Test set: Average loss: nan, Accuracy: 0/32 (0%)

S